In [1]:
from nltk.corpus import reuters
from nltk import ngrams
from nltk import FreqDist
from collections import Counter, defaultdict
import nltk
import re
import json
import pickle
import numpy as np
import math
import random
import glob, re
from itertools import product


სტანდარტულად წავიკითხოთ მონაცემები. იქიდან გამომდინარე რომ ქეგლზე პრობლემები შეგვექმნა მეხსიერების სიმცირის გამო, ვიყენებთ მონაცემთა დაახლოებით **მესამედს**

In [ ]:
docs = []
for ind, filename in enumerate(glob.glob("../data/whole-data/*")):
    if ind % 10000 == 0:
        print(ind, 'docs read')
    if ind == 100000:
        break
    fd = open(filename, 'r')
    text = fd.read() 
    docs.append(text)
    fd.close()
    
    
corpus = []
for doc in docs:
    doc = re.sub('(["-.,!?()])', r' \1 ', doc)
    tokens = doc.split()
    corpus.append(tokens)

corpus = [doc for doc in corpus if len(doc) > 25]

დავბეჭდოთ წაკითხული ტოკენები

In [ ]:
len(corpus), corpus[:10]
random.shuffle(corpus)

In [ ]:
train, test = corpus[:7000], corpus[7000:]
train[:10], test[:10]

ავაგოთ N-Gramის მოდელი. N-gram იყენებს სტატისტიკურ მონაცემებს ტექსტიდან და ცდილობს სიტყვების მიმდევრობათა სიხშირეების მიხედვით მომდევნო სიტყვის გამოცნობას.

In [ ]:
N = 4

In [ ]:
def preprocess(tokens, n):
    result = []
    for sent in tokens:
        sent = (n-1)*['<s>'] + sent + ['</s>']
        result += [sent]
    return result


დავა-flatten-ოთ მიღებული ტოკენები და ჩავამატოთ  დასაწყისის და დასასრულის აღმნიშვნელი ტოკენები

In [ ]:
train_data = preprocess(train, N)
flattened_tokens = [item for sublist in train_data for item in sublist]

test_data = preprocess(test, N)
test_flattened = [item for sublist in test_data for item in sublist]


In [ ]:
len(flattened_tokens)

In [ ]:
UNK = "<UNK>"


class NGram():
    """ნ-გრამის მოდელი მოცემული კორპუსისთვის

        აკეთებს ტექსტის პრეპროცესირებას, სიხშირეების დათვლას და ალბათობების ნორმალიზაციას.
        Args
            train_data (list of str): წინადედებების სია.
            n (int): ნ-გრამის n
            laplace (int): lambda multiplier to use for laplace smoothing (default 1 for add-1 smoothing).
    """
    def __init__(self, n, tokens, laplace=1):
        self.n = n
        self.laplace = laplace
        self.tokens = tokens 
        self.vocab = FreqDist(self.tokens)
        self.tokens = [token if self.vocab[token] > 1 else UNK for token in tokens]
        self.model = self._create_model()
        self.masks  = list(reversed(list(product((0,1), repeat=n))))

#         self.model = defaultdict(lambda: defaultdict(lambda: 1))
#         print(self.vocab)

    def _create_model(self):
        """
        ქმნის n-გრამ მოდელს მოცემული ტოკენებისთვის.
        m = n-1. იყენებს nltk-ის ngram ბიბლიოთეკას
        
        აბრუნებს ნ-gram - შეწონილი ალბათობების წყვილების dictionary-ს
        """
        vocab_size = len(self.vocab)

        n_grams = nltk.ngrams(self.tokens, self.n)
        n_vocab = nltk.FreqDist(n_grams)

        m_grams = nltk.ngrams(self.tokens, self.n-1)
        m_vocab = nltk.FreqDist(m_grams)

        def smoothed_count(n_gram, n_count):
            m_gram = n_gram[:-1]
            m_count = m_vocab[m_gram]
            return (n_count + self.laplace) / (m_count + self.laplace * vocab_size)

        return { n_gram: smoothed_count(n_gram, count) for n_gram, count in n_vocab.items() }
        

        
    def _convert_oov(self, ngram):
        """
        ანაცვებს მოდელისთვის უცნობ ნ-გრამს მისთვის ნაცნობი ნ-გრამით. 
        ამისავის UNK-ების მასკით თითოეულ წევრს ანაცვლებს UNK-ტოკენით.
        ყველაზე ცუდ შემთხვევაში 2^ნ მცდელობა.
        """
        mask = lambda ngram, bitmask: tuple((token if flag == 1 else "<UNK>" for token,flag in zip(ngram, bitmask)))

        ngram = (ngram,) if type(ngram) is str else ngram
        for possible_known in [mask(ngram, bitmask) for bitmask in self.masks]:
            if possible_known in self.model:
                return possible_known

            
    def best_candidate(self, prev, i):
        """
        აბრუნებს სავარაუდო გაგრძელებას ნგრამისა
        """
        candidates = list(((ngram[-1],prob) for ngram,prob in self.model.items() if ngram[:-1]==prev))
        strs = [ngram for ngram, prob in candidates]
        probs = [prob for ngram, prob in candidates]
        if len(strs) == 0:
            return ("</s>")
        index = random.choices(range(len(probs)), probs)
        return strs[index[0]]
        
    
    def generate_sent(self, min_len=12, max_len=24):
        sent, prob = ["<s>"] * max(1, self.n-1), 1
        while sent[-1] != "</s>":
            prev = () if self.n == 1 else tuple(sent[-(self.n-1):])
            blacklist = sent + (["</s>"] if len(sent) < min_len else [])
            next_token = self.best_candidate(prev, 1)
            sent.append(next_token)

            if len(sent) >= max_len:
                sent.append("</s>")
            
        return ' '.join(sent)

    
    def perplexity(self, test_tokens):
        """
        ითვლის მოდელის perpexity-ის ტესტ დატაზე
        
        """
        test_ngrams = nltk.ngrams(test_tokens, self.n)
        NN = len(test_tokens)

        known_ngrams  = (self._convert_oov(ngram) for ngram in test_ngrams)
        probabilities = [self.model[ngram] for ngram in known_ngrams]

        return math.exp((-1/NN) * sum(map(math.log, probabilities)))
        


In [ ]:
n_gram = NGram(N, flattened_tokens)

In [ ]:
def predict_next_word(sent, n):
    last_tokens = sent[-(n-1):]
#     print(last_tokens)
    next_word = n_gram.best_candidate(tuple(last_tokens), 1)
    return sent+[next_word]

დავწეროთ ფუნქცია, რომელიც აგრძელებს წინადადებას N ტოკენამდე

In [ ]:
def continue_sent(sent, max_len=14):
    for i in range (max_len):
        sent = (predict_next_word(sent, N))
        if sent[-1] == '</s>':
            return sent
    return sent

გავაგძელოთ მოცემული წინადადება:

In [ ]:
continue_sent(['მსოფლიო','საეკლესიო','კრების', ',', 'ეკლესიათა'], 6)

ფუნქცია რომელიც აგენერირებს შემთხვევით წინადადებას

In [ ]:
n_gram.generate_sent(max_len = 50)

მოდელის perplexity-ის დასათვლელად ვაგენერირებთ შემთხვევით წინადადებებს ტესტური მონაცემებიდან და ვამოწმებთ თუ სწორად აგრძელებს ჩვენი მოდელი. დამატებითი ინფორმაია იხილეთ რეპორტში

In [ ]:
n_gram.perplexity(test_flattened)